# Cleaning Sales dataset
#### by Jérôme d'Harveng

<a id='table'></a>
## Table of Contents
<ul>
<li><a href="#gathering">Gathering the data</a></li>
<li><a href="#assessing">Assessing</a></li>
<li><a href="#cleaning">Cleaning</a></li>
<li><a href="#exporting">Exporting to csv</a></li>
</ul>

<a id='gathering'></a>
## Gathering

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
retail_df = pd.read_excel('Data_P5/Online_Retail.xlsx')

##### Description of columns

> - **InvoiceNo:** Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 
> - **StockCode:** Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product. 
> - **Description:** Product (item) name. Nominal. 
> - **Quantity:** The quantities of each product (item) per transaction. Numeric.	
> - **InvoiceDate:** Invice Date and time. Numeric, the day and time when each transaction was generated. 
> - **UnitPrice:** Unit price. Numeric, Product price per unit in sterling. 
> - **CustomerID:** Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer. 
> - **Country:** Country name. Nominal, the name of the country where each customer resides.

In [3]:
retail_df.shape

(541909, 8)

In [4]:
retail_col = retail_df.columns.tolist()
print(retail_col)

['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


<a id='assessing'></a>
## Assessing

#### Make a copy to work on

In [5]:
# For the rest of the study hereunder we'll work on a COPY of the data for safety reasons
df_sales = retail_df.copy() 

### General Info

In [6]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
df_sales.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


> the minimum value of UnitPrice (-11062.06) seems suspect

### Visual inspection

In [8]:
df_sales.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [9]:
df_sales.tail(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [10]:
df_sales.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
192532,553423,22431,WATERING CAN BLUE ELEPHANT,2,2011-05-17 10:28:00,1.95,17811.0,United Kingdom
474924,576857,21990,MODERN FLORAL STATIONERY SET,1,2011-11-16 15:52:00,1.25,12748.0,United Kingdom
121487,546769,22251,BIRDHOUSE DECORATION MAGIC GARDEN,12,2011-03-16 14:57:00,1.25,17504.0,United Kingdom
514078,579673,23681,LUNCH BAG RED VINTAGE DOILY,1,2011-11-30 12:59:00,1.65,18283.0,United Kingdom
3100,536597,22197,SMALL POPCORN HOLDER,6,2010-12-01 17:35:00,0.85,18011.0,United Kingdom


### Missing Values

In [11]:
# Percentage of missing values
(df_sales.isnull().sum().sort_values(ascending=False)/df_sales.shape[0])*100

CustomerID     24.926694
Description     0.268311
Country         0.000000
UnitPrice       0.000000
InvoiceDate     0.000000
Quantity        0.000000
StockCode       0.000000
InvoiceNo       0.000000
dtype: float64

> - CustomerID: has almost 25% of missing values
> - Description: has less than 1% (but no missing values for StockCode => that's the most important

### Duplicates

In [12]:
print('Amount of duplicated lines :',df_sales.duplicated().sum())
print('% of duplicated lines :',(df_sales.duplicated().sum()/df_sales.shape[0])*100)

Amount of duplicated lines : 5268
% of duplicated lines : 0.9721189350979592


#### Finding some examples

In [13]:
df_sales[df_sales.duplicated()].head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom


In [14]:
df_sales[df_sales.duplicated()].tail(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541701,581538,23343,JUMBO BAG VINTAGE CHRISTMAS,1,2011-12-09 11:34:00,2.08,14446.0,United Kingdom


In [15]:
df_sales.query('InvoiceNo==536409 and StockCode==21866 and Quantity==1 and UnitPrice==1.25 and CustomerID==17908.0')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom


In [16]:
df_sales.query('InvoiceNo==581538 and StockCode==23343 and Quantity==1 and UnitPrice==2.08 and CustomerID==14446.0')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541666,581538,23343,JUMBO BAG VINTAGE CHRISTMAS,1,2011-12-09 11:34:00,2.08,14446.0,United Kingdom
541701,581538,23343,JUMBO BAG VINTAGE CHRISTMAS,1,2011-12-09 11:34:00,2.08,14446.0,United Kingdom


> There are 5268 duplicates lines

### Unique Values

In [17]:
unique_col = ['InvoiceNo', 'StockCode', 'InvoiceDate','CustomerID', 'Country']

In [18]:
# A mettre dans l'exploration !!!!!
(df_sales['Country'].value_counts()[:5]/df_sales.shape[0])*100
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

United Kingdom    91.431956
Germany            1.752139
France             1.579047
EIRE               1.512431
Spain              0.467422
Name: Country, dtype: float64

In [19]:
print(df_sales.Country.unique())

['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']


##### There's one value: Unspecified

In [20]:
# how many lines are concerned?
print(df_sales.query('Country=="Unspecified"').shape)
df_sales.query('Country=="Unspecified"').head(2)

(446, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
152712,549687,20685,DOORMAT RED RETROSPOT,2,2011-04-11 13:29:00,7.95,12363.0,Unspecified
152713,549687,22691,DOORMAT WELCOME SUNRISE,2,2011-04-11 13:29:00,7.95,12363.0,Unspecified


In [21]:
df_sales.query('CustomerID==12363.0 and Country!="Unspecified"')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [22]:
for col_name in unique_col:
    print(f'{col_name} has that amount of distinct values :' ,df_sales[col_name].nunique())

InvoiceNo has that amount of distinct values : 25900
StockCode has that amount of distinct values : 4070
InvoiceDate has that amount of distinct values : 23260
CustomerID has that amount of distinct values : 4372
Country has that amount of distinct values : 38


### InvoiceDate

In [23]:
print(df_sales.InvoiceDate.min())
print(df_sales.InvoiceDate.max())

2010-12-01 08:26:00
2011-12-09 12:50:00


### UnitPrice

#### UnitPrice < 0

In [24]:
print(df_sales.query('UnitPrice < 0').shape[0])
df_sales.query('UnitPrice < 0')

2


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


> There are only 2 transactions with negative prices

#### UnitPrice == 0

In [25]:
print(df_sales.query('UnitPrice == 0').shape[0])
df_sales.query('UnitPrice == 0').sample(5)

2515


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
387141,570263,23412,NaN,2,2011-10-10 10:00:00,0.0,NaN,United Kingdom
116357,546240,85045,NaN,20,2011-03-10 13:00:00,0.0,NaN,United Kingdom
320587,565047,21257,NaN,7,2011-08-31 17:04:00,0.0,NaN,United Kingdom
142584,548615,16010,NaN,-7,2011-04-01 11:47:00,0.0,NaN,United Kingdom
479163,577145,20832,check,110,2011-11-18 09:35:00,0.0,NaN,United Kingdom


#### UnitPrice == 0 and depending  Quantity 

In [26]:
print(df_sales.query('UnitPrice == 0 and Quantity < 0').shape[0])
df_sales.query('UnitPrice == 0 and Quantity < 0').sample(3)

1336


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
242057,558307,22088,NaN,-39,2011-06-28 11:02:00,0.0,NaN,United Kingdom
381675,569874,90180A,stock check,-32,2011-10-06 15:06:00,0.0,NaN,United Kingdom
171573,551429,10002,NaN,-3,2011-04-28 15:05:00,0.0,NaN,United Kingdom


In [27]:
print(df_sales.query('UnitPrice == 0 and Quantity > 0').shape[0])
df_sales.query('UnitPrice == 0 and Quantity > 0').sample(3)

1179


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
80659,543049,79063C,NaN,320,2011-02-03 10:09:00,0.0,NaN,United Kingdom
51758,540674,22837,Found,26,2011-01-10 16:05:00,0.0,NaN,United Kingdom
242417,558340,22678,FRENCH BLUE METAL DOOR SIGN 3,2,2011-06-28 14:01:00,0.0,NaN,United Kingdom


In [28]:
print(df_sales.query('UnitPrice == 0 and Quantity > 0 and not(CustomerID.isnull())').shape[0])
df_sales.query('UnitPrice == 0 and Quantity > 0 and not(CustomerID.isnull())').sample(3)

40


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
454463,575579,22437,SET OF 9 BLACK SKULL BALLOONS,20,2011-11-10 11:49:00,0.0,13081.0,United Kingdom
282912,561669,22960,JAM MAKING SET WITH JARS,11,2011-07-28 17:09:00,0.0,12507.0,Spain
314748,564651,21786,POLKADOT RAIN HAT,144,2011-08-26 14:19:00,0.0,14646.0,Netherlands


### InvoiceNo

> - **InvoiceNo:** Invoice number. Nominal, a **6-digit integral number** uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 

#### Let's have a look at Cancelled and Adapted Invoices

In [29]:
df_sales.query('InvoiceNo.str.len()>6').shape[0]

9291

##### Adapted Invoices

In [30]:
print("Amount of adapted invoice : ", df_sales.query('InvoiceNo.str[0] == "A"').shape[0])
df_sales.query('InvoiceNo.str[0] == "A"')

Amount of adapted invoice :  3


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


##### Cancelled invoices

In [31]:
print("Amount of cancelled invoice : ", df_sales.query('InvoiceNo.str[0] == "C"').shape[0])
df_sales.query('InvoiceNo.str[0] == "C"').sample(3)

Amount of cancelled invoice :  9288


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
440070,C574503,23567,EGG CUP HENRIETTA HEN PINK,-1,2011-11-04 13:19:00,1.25,16191.0,United Kingdom
301378,C563258,22061,LARGE CAKE STAND HANGING STRAWBERY,-2,2011-08-15 12:08:00,8.50,15311.0,United Kingdom
101135,C544902,22629,SPACEBOY LUNCH BOX,-1,2011-02-24 13:05:00,1.95,12362.0,Belgium


## StockCode

> - **StockCode:** Product (item) code. Nominal, a **5-digit integral** number uniquely assigned to each distinct 

#### Let's now have a look at special StockCodes

In [32]:
print("Amount of StockCode with more than 5 digits: ",df_sales.query('StockCode.str.len() > 5').shape[0])
print("Amount of UNIQUE special StockCodes with more than 5 digits: ",
      df_sales.query('StockCode.str.len() > 5').StockCode.unique().shape[0])
df_sales.query('StockCode.str.len() > 5').sample(3)

Amount of StockCode with more than 5 digits:  52028
Amount of UNIQUE special StockCodes with more than 5 digits:  1114


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
10793,537237,90018A,SILVER M.O.P ORBIT DROP EARRINGS,1,2010-12-06 09:58:00,4.24,NaN,United Kingdom
131788,547649,15056P,EDWARDIAN PARASOL PINK,1,2011-03-24 12:02:00,5.95,18118.0,United Kingdom
306647,563830,47348A,FUSCHIA VOILE POINTY SHOE DEC,12,2011-08-19 12:20:00,0.39,16638.0,United Kingdom


In [33]:
print("Amount of StockCode with less than 5 digits: ",df_sales.query('StockCode.str.len() < 5').shape[0])
print("List of special StockCodes: ",list(df_sales.query('StockCode.str.len() < 5').StockCode.unique()))
df_sales.query('StockCode.str.len() < 5').sample(3)

Amount of StockCode with less than 5 digits:  2845
List of special StockCodes:  ['POST', 'D', 'C2', 'DOT', 'M', 'S', 'm', 'PADS', 'B', 'CRUK']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
181009,C552423,D,Discount,-1,2011-05-09 12:44:00,11.84,14527.0,United Kingdom
233799,557489,POST,POSTAGE,3,2011-06-20 14:58:00,18.00,12490.0,France
134704,547863,POST,POSTAGE,2,2011-03-27 13:07:00,18.00,12728.0,France


In [34]:
df_sales.query('StockCode.str.len() < 5').StockCode.value_counts()

POST    1256
DOT      710
M        571
C2       144
D         77
S         63
CRUK      16
PADS       4
B          3
m          1
Name: StockCode, dtype: int64

In [35]:
df_sales.query('StockCode.str.len() < 5').Description.value_counts()

POSTAGE                       1252
DOTCOM POSTAGE                 709
Manual                         572
CARRIAGE                       143
Discount                        77
SAMPLES                         63
CRUK Commission                 16
PADS TO MATCH ALL CUSHIONS       4
Adjust bad debt                  3
Name: Description, dtype: int64

### First observations:
>- Total amount of products : 541909


>- **Missing values** => null values:
    * CustomerID: has almost 25% of missing values
    * Description: has less than 1% (but no missing values for StockCode => that's the most important  
>
> - **Duplicates**: 
>   * There 5268 duplicated lines
>
>- **Wrong values**:
>    * For the segmentation purpose, we'll keep only positive values for Quantity and UnitPrice
>    * The are 446 lines with as value four Country : "Unspecified"
>
>- **Types**: 
>    * CustomerID: is in 'float' should be 'int' (or string)    

<a id='cleaning'></a>
## Cleaning

<a href="#table">Table of Content</a>

#### 1. Handling missing values : CustomerId

##### Define

As the final goal of the investigation, is to study customer behaviour, CustomerID is very important. So we decided to drop the lines having missing values for CustomerID.

##### Code

In [36]:
df_sales_clean = df_sales.dropna(subset=['CustomerID'])

##### Test

In [37]:
print("Amount of null values before dropping missing values : ",df_sales.CustomerID.isnull().sum())
print("Amount of null values after dropping missing values : ",df_sales_clean.CustomerID.isnull().sum())

Amount of null values before dropping missing values :  135080
Amount of null values after dropping missing values :  0


In [38]:
print("Size of df_sales_clean :  ", df_sales_clean.shape)
df_sales_clean.isnull().sum()

Size of df_sales_clean :   (406829, 8)


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

#### 2. Dropping Duplicates

##### Define

We saw that there were some duplicated lines, so we will suppress those.

##### Code

In [39]:
df_sales_clean = df_sales_clean.drop_duplicates()

##### Test

In [40]:
print(df_sales_clean.shape)
print("Amount of duplicated lines after dropping them: ",df_sales_clean.duplicated().sum())

(401604, 8)
Amount of duplicated lines after dropping them:  0


#### 3. Delete lines with Country == "Unspecified"

##### Define

As the original Country can't be found back and that there are only a few lines affected, the best is to delete those.

##### Code

In [41]:
df_sales_clean = df_sales_clean.query('Country!= "Unspecified"')

##### Test

In [42]:
print(df_sales_clean.Country.unique())
df_sales_clean.query('Country == "Unspecified"').shape

['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Greece' 'Singapore' 'Lebanon'
 'United Arab Emirates' 'Saudi Arabia' 'Czech Republic' 'Canada' 'Brazil'
 'USA' 'European Community' 'Bahrain' 'Malta' 'RSA']


(0, 8)

#### 4. Type of CustomerID

##### Define

We will convert CustomerID from float to int

##### Code

In [43]:
print(df_sales_clean.CustomerID.dtypes)
df_sales_clean['CustomerID'] = df_sales_clean['CustomerID'].astype('int')

float64


##### Test

In [44]:
df_sales_clean.CustomerID.dtypes

dtype('int64')

#### 5. FEATURE ENGINEERING: Adding total value (qty x unitprice)

##### Define

As in the customer segmentation, the total customer spent will be important, we'll start here to add an extra column with "Amount" = Quantity x UnitPrice

##### Code

In [45]:
df_sales_clean['Amount'] = df_sales_clean['Quantity']*df_sales_clean['UnitPrice']

##### Test

In [46]:
print(2.55*6)
print(3.39*6)
print(2.75*8)
df_sales_clean.head(3)

15.299999999999999
20.34
22.0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00


#### 6. Adding column Day (month, week) and hour extracted from InvoiceDate

##### Define

For later analysis and purpose we'll add columns extracted from the InvoiceDate

##### Code

In [47]:
df_sales_clean['InvoiceDate'] = pd.to_datetime(df_sales_clean['InvoiceDate'])

In [48]:
df_sales_clean['Month'] = df_sales_clean['InvoiceDate'].apply(lambda x: x.month)
df_sales_clean['DayOfMonth'] = df_sales_clean['InvoiceDate'].apply(lambda x: x.day)
df_sales_clean['DayOfWeek'] = df_sales_clean['InvoiceDate'].apply(lambda x: x.isoweekday())
df_sales_clean['Hour'] = df_sales_clean['InvoiceDate'].apply(lambda x: x.hour)

##### Test

In [49]:
df_sales_clean.sample(5)[['InvoiceNo','InvoiceDate','Month','DayOfMonth','DayOfWeek','Hour']]

,InvoiceNo,InvoiceDate,Month,DayOfMonth,DayOfWeek,Hour
102977,545045,2011-02-27 11:40:00,2,27,7,11
200689,554151,2011-05-23 11:07:00,5,23,1,11
190864,553214,2011-05-16 09:49:00,5,16,1,9
342970,566922,2011-09-15 14:58:00,9,15,4,14
494363,578270,2011-11-23 13:39:00,11,23,3,13


#### 7. Wrong values for segmenation afterwards

##### Define

- For the exploration we'll keep the cancelled invoices
- But for the customer segmentation afterwards, we'll only invoices with positive UnitPrices and Quanities
- For the segmentation we'll also remove the special StockCodes (less than 5 digits)

##### Code

In [55]:
df_segmentation = df_sales_clean.query('Quantity>0 and UnitPrice>0')

In [56]:
df_segmentation.query('StockCode.str.len() < 5').Description.value_counts()

POSTAGE                       1099
Manual                         279
CARRIAGE                       133
DOTCOM POSTAGE                  16
PADS TO MATCH ALL CUSHIONS       3
Name: Description, dtype: int64

In [57]:
df_segmentation = df_segmentation.query('not(StockCode.str.len() < 5)')

##### Test

In [58]:
print("Size df_sales_clean :", df_sales_clean.shape)
print("Size df_segmentation : ", df_segmentation.shape)

Size df_sales_clean : (401363, 13)
Size df_segmentation :  (390921, 13)


In [59]:
print(df_segmentation.query('Quantity <=0').shape[0])
print(df_segmentation.query('UnitPrice <=0').shape[0])
print(df_segmentation.Quantity.min())
print(df_segmentation.UnitPrice.min())

0
0
1
0.001


In [60]:
print(df_segmentation.shape)
df_segmentation.query('StockCode.str.len() < 5').Description.value_counts()

(390921, 13)


Series([], Name: Description, dtype: int64)

<a id='exporting'></a>
### Exporting Cleaned data in csv
<a href="#table">Back to table of content</a>

In [61]:
# Cleaned dataset BUT STILL with cancellation dataset
df_sales_clean.to_csv('df_sales_clean.csv', encoding='utf_8', index=False)

In [62]:
# Cleaned dataset READY for SEGMENTATION (= WITHOUT negative Quantities and negative prices)
df_segmentation.to_csv('df_segmentation.csv', encoding='utf_8', index=False)

###  Exporting for some samples to test the final Python Code 

In [63]:
df_segmentation.CustomerID.sample(3)

324443    16686
208056    16483
165532    14334
Name: CustomerID, dtype: int64

In [67]:
colums_export = ['CustomerID', 'Country','InvoiceDate','InvoiceNo', 'StockCode', 'Description',
                 'Quantity', 'UnitPrice']


In [74]:
df_segmentation.query('CustomerID==14334')[colums_export].sample(1)\
                                .to_csv('df_test1.csv', encoding='utf_8', index=False)


In [75]:
df_segmentation.query('CustomerID==16483')[colums_export].sample(5)\
                                .to_csv('df_test5.csv', encoding='utf_8', index=False)

In [76]:
df_segmentation.query('CustomerID==16686').sample(15)[colums_export]\
                                .to_csv('df_test15.csv', encoding='utf_8', index=False)